In [5]:
import csv
import pandas as pd
def convert_csv_time_values(input_file_path, output_file_path):
    time_columns = ['Validate', 'Provision', 'Snr', 'Tear down', 'Evaluate']
    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w', newline='') as output_file:
        reader = csv.reader(input_file, delimiter=';')
        writer = csv.writer(output_file, delimiter=';')
        headers = next(reader)
        writer.writerow(headers)
        time_indices = [headers.index(col) for col in headers if col in time_columns]

        for row in reader:
            for index in time_indices:
                time_parts = row[index].split(' ')
                total_seconds = 0
                for part in time_parts:
                    if 'm' in part:
                        total_seconds += int(part[:-1]) * 60
                    elif 's' in part:
                        total_seconds += int(part[:-1])
                row[index] = str(total_seconds)
            writer.writerow(row)


In [6]:
input_file_path = './workflow_duration.csv'
output_file_path = './workflow_dur_sec.csv'
convert_csv_time_values(input_file_path, output_file_path)


In [7]:
def split_csv_to_dataframes(csv_file_path):
    df = pd.read_csv(csv_file_path, delimiter=';')
    mongo_df = df[df['DB'] == 'mongo']
    mysql_df = df[df['DB'] == 'mysql']

    return mongo_df, mysql_df

csv_file_path = './workflow_dur_sec.csv'
mongo_df, mysql_df = split_csv_to_dataframes(csv_file_path)



In [8]:
mongo_df

,ID,DB,Validate,Provision,Snr,Tear down,Evaluate
0,1,mongo,7,125,288,139,46
1,2,mongo,8,146,258,120,38
2,3,mongo,6,134,256,126,30
3,4,mongo,10,315,276,137,34
4,5,mongo,4,113,293,123,41
5,6,mongo,6,112,283,124,33
7,8,mongo,8,113,284,128,40
8,9,mongo,5,145,277,110,30
9,10,mongo,7,143,252,138,29
12,13,mongo,4,281,228,136,28


In [9]:
mysql_df

,ID,DB,Validate,Provision,Snr,Tear down,Evaluate
6,7,mysql,5,112,425,115,29
10,11,mysql,5,130,346,214,37
11,12,mysql,7,151,340,127,34
14,15,mysql,9,155,363,222,36
15,16,mysql,6,112,366,119,35
19,20,mysql,8,172,419,121,37
20,21,mysql,7,141,304,137,35
21,22,mysql,8,135,309,129,29
24,25,mysql,10,150,313,125,28
25,26,mysql,8,163,346,126,34


In [10]:
def calculate_statistics(df):
    columns = ['Validate', 'Provision', 'Snr', 'Tear down', 'Evaluate']
    statistics = {}
    for col in columns:
        mean = df[col].mean()
        std = df[col].std()
        statistics[col] = {'mean': mean, 'std': std}
    return statistics

mongo_stats = calculate_statistics(mongo_df)
mysql_stats = calculate_statistics(mysql_df)

mongo_stats, mysql_stats

({'Validate': {'mean': 6.761904761904762, 'std': 1.894855189843327},
  'Provision': {'mean': 150.47619047619048, 'std': 51.30654836141196},
  'Snr': {'mean': 270.04761904761904, 'std': 19.497374670647815},
  'Tear down': {'mean': 132.52380952380952, 'std': 21.53048779665488},
  'Evaluate': {'mean': 33.714285714285715, 'std': 5.80640040940045}},
 {'Validate': {'mean': 7.166666666666667, 'std': 1.6422453217986943},
  'Provision': {'mean': 137.75, 'std': 20.89421407514965},
  'Snr': {'mean': 352.5833333333333, 'std': 38.232442989818175},
  'Tear down': {'mean': 140.5, 'std': 36.68663368188871},
  'Evaluate': {'mean': 34.833333333333336, 'std': 4.58918359412718}})